In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
import ipywidgets as widgets
from IPython.display import display

warnings.filterwarnings("ignore", category=FutureWarning)

# define custom functions

In [ ]:
def fill_missing_dates(df):
    start_date = df['order_date'].min()
    end_date = df['order_date'].max()
    categories = df['categ'].unique()
    full_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    filled_df = pd.DataFrame()
    for category in categories:
        cat_data = df[df['categ'] == category].copy()
        cat_data.set_index('order_date', inplace=True)
        cat_data.index = pd.to_datetime(cat_data.index)
        cat_data = cat_data.reindex(full_dates, fill_value=0).asfreq('D')
        cat_data['categ'] = category
        filled_df = pd.concat([filled_df, cat_data])
    
    filled_df.reset_index(inplace=True)
    filled_df.rename(columns={'index': 'order_date'}, inplace=True)
    return filled_df

def create_mae_widget(mae_df):
    def display_mae(category):
        mae_value = mae_df[mae_df['category'] == category]['MAE'].values[0]
        print(f'MAE for {category}: {mae_value}')

    category_dropdown = widgets.Dropdown(
        options=mae_df['category'].tolist(),
        description='Category:',
        disabled=False)

    interactive_mae = widgets.interactive(display_mae, category=category_dropdown)
    display(interactive_mae)

def create_forecast_widget(forecast_df):
    def display_forecast(category):
        filtered_df = forecast_df[forecast_df['categ'] == category]
        display(filtered_df)

    category_dropdown = widgets.Dropdown(
        options=forecast_df['categ'].unique().tolist(),
        description='Category:',
        disabled=False)

    interactive_forecast = widgets.interactive(display_forecast, category=category_dropdown)
    display(interactive_forecast)


# Load data

In [ ]:
demand_data = pd.read_csv('/Users/nataliamarko/Documents/jobs_applications/test_Smart_business/S_Data/category_data.csv')
print(demand_data.shape)
df = demand_data
df

In [ ]:
df[df['categ']=='agro_industry_and_commerce']

## fill missed dates

In [ ]:
df = fill_missing_dates(df)
df_filled_days_onecateg = df[df['categ'] == 'agro_industry_and_commerce']
df_filled_days_onecateg

In [ ]:
cat = 'agro_industry_and_commerce'
filtered_data = df[df['categ'] == cat]

plt.plot(filtered_data['order_date'], filtered_data['demand'], marker='o', linestyle='-', color='b')
plt.title(f'Demand over Time for {cat}')
plt.xlabel('Date')
plt.ylabel('Demand')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## looking for seasonality, p,d,q

In [ ]:
from scipy.fft import fft
from statsmodels.tsa.seasonal import seasonal_decompose

def find_domin_freq(df):

    decomposition = seasonal_decompose(df, model='additive')

    # Extract the seasonal values
    seasonal = decomposition.seasonal

    # Compute the FFT and frequencies
    fft_values = fft(seasonal.values)
    frequencies = np.fft.fftfreq(len(fft_values))

    # Compute the absolute values to get the magnitudes
    fft_magnitudes = np.abs(fft_values)

    # Find the index of the peak frequency
    # Note: We ignore the first element which is the zero-frequency component
    peak_frequency_index = np.argmax(fft_magnitudes[1:]) + 1

    # The actual frequency is then
    peak_frequency = frequencies[peak_frequency_index]

    # # Plot the FFT spectrum
    # plt.figure(figsize=(14, 5))
    # plt.stem(frequencies[1:], fft_magnitudes[1:], 'b', markerfmt=" ", basefmt="-b")
    # plt.title('FFT Spectrum')
    # plt.xlabel('Frequency')
    # plt.ylabel('Magnitude')
    # plt.show()

    # print(f"The dominant frequency is: {peak_frequency}")

    s = int(1/peak_frequency)

    return s

In [ ]:
df

In [ ]:
df['order_date'] = pd.to_datetime(df['order_date'])
# Get the list of unique categories
categories = df['categ'].unique()

for category in categories:
    df_category = df[df['categ'] == category].copy()
    df_category.set_index('order_date', inplace=True)
    s = find_domin_freq(df_category['demand'])
    print(f's in category {category}:', s)

In [ ]:
df_test = filtered_data.copy()
df_test = df_test.set_index('order_date').drop('categ', axis=1)
df_test = df_test.asfreq('D').fillna(0)
df_test

In [ ]:
decomposition = seasonal_decompose(df_test, model='additive')

# Extract the components
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Plotting the components
plt.figure(figsize=(10, 7))

plt.subplot(411)
plt.plot(df_test['demand'], label='Original', color='blue')
plt.title('Original Time Series')
plt.legend(loc='upper left')

plt.subplot(412)
plt.plot(trend, label='Trend', color='orange')
plt.title('Trend Component')
plt.legend(loc='upper left')

plt.subplot(413)
plt.plot(seasonal, label='Seasonal', color='green')
plt.title('Seasonal Component')
plt.legend(loc='upper left')

plt.subplot(414)
plt.plot(residual, label='Residual', color='red')
plt.title('Residual Component')
plt.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(df_test['demand'], lags=40, ax=plt.gca(), title='Partial Autocorrelation Function (PACF)')
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(df_test['demand'], lags=40, ax=plt.gca(), title='Autocorrelation Function (ACF)')
plt.show()


## feature engineering

In [ ]:
# add lags and rolling mean(), std()
def add_features(df):
    for lag in [1, 2, 3, 7]:
        df[f'lag_{lag}'] = df.groupby('categ')['demand'].shift(lag)
    df['rolling_mean_7'] = df.groupby('categ')['demand'].shift(7).rolling(window=7).mean()
    df['rolling_std_7'] = df.groupby('categ')['demand'].shift(7).rolling(window=7).std()
    df.fillna(0, inplace=True)
    return df

df = add_features(df)
df[df['categ'] == cat]

## train data

In [ ]:
# Split data into train and test
def split_train_test(df, test_size=90):
    train = df.iloc[:-test_size]
    test = df.iloc[-test_size:]
    return train, test

# Define the function to forecast demand and calculate MAE
def forecast_demand(df, category, seasonal_order):
    try:
        # Filter data for the specific category
        cat_data = df[df['categ'] == category].copy()

        # Prepare the data
        cat_data.set_index('order_date', inplace=True)
        cat_data.index = pd.to_datetime(cat_data.index).to_period('D')
        
        # Split into train and test
        train, test = split_train_test(cat_data)
        
        # Ensure there are enough observations in the training set
        if len(train) < 30:
            print(f"Skipping category {category} due to insufficient training data")
            return None, None
        
        # Define the SARIMAX model with exogenous variables
        exog_vars = ['lag_1', 'lag_2', 'lag_3', 'lag_7', 'rolling_mean_7', 'rolling_std_7']
        model = SARIMAX(train['demand'], 
                        exog=train[exog_vars],
                        order=(1, 1, 1), 
                        seasonal_order=seasonal_order,
                        enforce_stationarity=False, 
                        enforce_invertibility=False)
        
        results = model.fit(maxiter=500, pgtol=1e-3, method='lbfgs', disp=False)

        # Forecast on test data
        test_exog = test[exog_vars]
        forecast = results.get_forecast(steps=len(test), exog=test_exog)
        forecast_df = forecast.summary_frame()
        
        # Calculate MAE
        mae = mean_absolute_error(test['demand'], forecast_df['mean'])
        
        # Prepare forecast dataframe for future dates
        future_dates = pd.date_range(start=cat_data.index[-1].to_timestamp() + pd.Timedelta(days=1), periods=21, freq='D')
        
        # Create future exogenous variables for forecasting
        last_obs = cat_data.iloc[-1]
        future_exog = pd.DataFrame(index=future_dates.to_period('D'))
        future_exog['lag_1'] = last_obs['demand']
        future_exog['lag_2'] = cat_data['demand'].iloc[-2] if len(cat_data) > 1 else 0
        future_exog['lag_3'] = cat_data['demand'].iloc[-3] if len(cat_data) > 2 else 0
        future_exog['lag_7'] = cat_data['demand'].iloc[-7] if len(cat_data) > 6 else 0
        future_exog['rolling_mean_7'] = cat_data['demand'].iloc[-7:].mean()
        future_exog['rolling_std_7'] = cat_data['demand'].iloc[-7:].std()
        
        # Forecast for the next 14 days with a 7 days gap 
        future_forecast = results.get_forecast(steps=21, exog=future_exog)
        future_forecast_df = future_forecast.summary_frame()
        
        # Create a DataFrame for future dates
        future_df = pd.DataFrame(index=future_dates)
        future_df['demand'] = future_forecast_df['mean'].values
        future_df.reset_index(inplace=True)
        future_df.rename(columns={'index': 'order_date'}, inplace=True)
        future_df['categ'] = category

        return future_df, mae

    except Exception as e:
        print(f"Error in forecasting demand for category {category}: {e}")
        return None, None

## train & predict

In [ ]:
# Define the seasonal order
seasonal_order = (1, 1, 1, 30)

# Get the list of unique categories
categories = df['categ'].unique()

# Initialize dictionaries to store results
forecasts = {}
mae_list = []

# Loop through each category and forecast
for category in categories:
    forecast_result, test_mae = forecast_demand(df, category, seasonal_order)
    if forecast_result is not None:
        forecasts[category] = forecast_result
        mae_list.append({'category': category, 'MAE': round(test_mae, 3)})

In [ ]:
# Combine forecasts into a single DataFrame
if forecasts:
    forecast_df = pd.concat(forecasts.values(), ignore_index=True)
    forecast_df = forecast_df[['order_date', 'demand', 'categ']]
    forecast_df['demand'] = forecast_df['demand'].apply(lambda x: max(x, 0)).round(0)
    ahead_14_days_forecast = forecast_df.groupby('categ').tail(14)
    all_forecast = ahead_14_days_forecast.groupby('order_date').sum().reset_index()
    all_forecast['categ'] = 'all'
    ahead_14_days_forecast = pd.concat([ahead_14_days_forecast, all_forecast], ignore_index=True)
    print(ahead_14_days_forecast)
else:
    print("No valid forecasts generated.")

# Print MAE for each category
mae_df = pd.DataFrame(mae_list)
mae_df

## evaluate the mdoel

In [ ]:
# run and select the category to display the mae(mean absolute error) for each category
create_mae_widget(mae_df)

In [ ]:
df['order_date'].max()

## forecasting ahead 14 days with a 7day lap

In [ ]:
# run and select the category to display the forecast for each category
create_forecast_widget(ahead_14_days_forecast)